## NeXtCortex project

The installation of the packages DrWatson, SpikingNeuralNetworks, UnPack, Logging, Plots, Statistics, Random, CSV, DataFrames, Unitful are required to run this file.

The file src/networks_utils.jl is also required.

In [ ]:
using DrWatson
findproject(@__DIR__) |> quickactivate

include("src/network_utils.jl")
using .NetworkUtils

using SpikingNeuralNetworks
using UnPack
using Logging
using Plots
using Statistics
using Random
using CSV
using DataFrames
import SpikingNeuralNetworks: @update

global_logger(NullLogger())
SNN.@load_units

import SpikingNeuralNetworks: PoissonLayer, monitor!, sim!, SingleExpSynapse, IFParameter, PostSpike, STTC

img_path = "plots";

#### Network baseline configuration

The baseline network was built from a pre-existing implementation.Parameter values were chosen within commonly used ranges.

In [ ]:
TC3inhib_network = (
    # Number of neurons in each population
    Npop = (TE=200, CE=4000, PV=400, SST=300, VIP=300),
    seed = 1234,

    # Parameters for cortical and thalamic excitatory neurons
    exc = IFParameter(
        τm=200pF / 10nS,  # Membrane time constant
        El=-70mV,         # Leak reversal potential
        Vt=-50.0mV,       # Spike threshold
        Vr=-70.0f0mV,     # Reset potential
        R=1 / 10nS,       # Membrane resistance
    ),

    # Parameters for the different populations of inhibitory neurons
    inh_PV  = IFParameter(τm=100pF/10nS, El=-70mV, Vt=-53mV, Vr=-70mV, R=1/10nS),
    inh_SST = IFParameter(τm=200pF/10nS, El=-70mV, Vt=-53mV, Vr=-70mV, R=1/10nS),
    inh_VIP = IFParameter(τm=150pF/10nS, El=-70mV, Vt=-53mV, Vr=-70mV, R=1/10nS),

    # Spiking threshold properties: absolute refractory period
    spike     = PostSpike(τabs=5ms),
    spike_PV  = PostSpike(τabs=2ms),
    spike_SST = PostSpike(τabs=10ms),
    spike_VIP = PostSpike(τabs=5ms),

    # Synaptic properties
    synapse=SingleExpSynapse(
        τi=5ms,             # Inhibitory synaptic time constant
        τe=5ms,             # Excitatory synaptic time constant
        E_i=-80mV,          # Inhibitory reversal potential
        E_e=0mV             # Excitatory reversal potential
    ),
    synapse_PV  = SingleExpSynapse(τi=3ms, τe=5ms, E_i=-80mV, E_e=0mV),
    synapse_SST = SingleExpSynapse(τi=12ms, τe=5ms, E_i=-80mV, E_e=0mV),
    synapse_VIP = SingleExpSynapse(τi=7ms, τe=5ms, E_i=-80mV, E_e=0mV),

    # Connection probabilities and synaptic weights
    connections = (
        TE_to_CE = (p=0.05, μ=4nS, rule=:Fixed),
        TE_to_PV = (p=0.05, μ=4nS, rule=:Fixed),

        CE_to_CE = (p=0.05, μ=2nS, rule=:Fixed),
        CE_to_PV = (p=0.05, μ=2nS, rule=:Fixed),
        CE_to_TE = (p=0.05, μ=2nS, rule=:Fixed),
        CE_to_SST = (p=0.05, μ=2nS, rule=:Fixed),
        CE_to_VIP = (p=0.05, μ=2nS, rule=:Fixed),

        PV_to_CE  = (p=0.05, μ=10nS, rule=:Fixed),
        PV_to_PV  = (p=0.05, μ=10nS, rule=:Fixed),
        PV_to_SST = (p=0.05, μ=10nS, rule=:Fixed),

        SST_to_CE  = (p=0.025, μ=10nS, rule=:Fixed),
        SST_to_PV  = (p=0.025, μ=10nS, rule=:Fixed),
        SST_to_VIP = (p=0.025, μ=10nS, rule=:Fixed),

        VIP_to_SST = (p=0.3, μ=10nS, rule=:Fixed),
    ),

    # Parameters for external Poisson input
    afferents_to_TE=(
        layer=PoissonLayer(rate=1.5Hz, N=1000),           # Poisson input layer
        conn=(p=0.05, μ=4nS, rule=:Fixed),              # Connection probability and weight
    ),
    afferents_to_CE  = (layer=PoissonLayer(rate=1.5Hz, N=1000), conn=(p=0.02, μ=4nS, rule=:Fixed)),
    afferents_to_PV  = (layer=PoissonLayer(rate=1.5Hz, N=1000), conn=(p=0.02, μ=4nS, rule=:Fixed)),
    afferents_to_SST = (layer=PoissonLayer(rate=1.5Hz, N=1000), conn=(p=0.15, μ=2nS, rule=:Fixed)),
    afferents_to_VIP = (layer=PoissonLayer(rate=1.5Hz, N=1000), conn=(p=0.10, μ=2nS, rule=:Fixed)),
);

In [ ]:
# Build and simulate
model = NetworkUtils.build_network(TC3inhib_network)
monitor!(model.pop, [:v], sr=1kHz)


Random.seed!(TC3inhib_network.seed)
sim!(model, 3s);

In [ ]:
rplt, zrplt, frplt, vplts = NetworkUtils.plot_analysis(model, img_path)

#Display of the baseline network activity
plts = plot(rplt, zrplt, frplt, vplt, layout=(2,2), size=(1000,1000))
display(plts)
savefig(plts, "$img_path/Baseline tight_layout.png")

#### Slower inhibition test (increasing PV inhibitory synaptic time constant)

One of the first test we did was incresing the inhibitory synaptic time constant to slow inhibition. However we have seen that slowing done inhibition incresed the inhibition.

In [ ]:
TC3inhib_network_modified = (; TC3inhib_network..., 
    synapse_PV  = SingleExpSynapse(τi=25ms, τe=5ms, E_i=-80mV, E_e=0mV),
)

model = NetworkUtils.build_network(TC3inhib_network_modified)
monitor!(model.pop, [:v], sr=1kHz)
Random.seed!(TC3inhib_network_modified.seed)
sim!(model, 3s)

rplt, zrplt, frplt, vplt = NetworkUtils.plot_analysis(model, img_path, name = "Slower inhibition")

plts = plot(rplt, zrplt, frplt, vplt, layout=(2,2), size=(1000,1000))
display(plts)
savefig(plts, "$img_path/Slower inhibition tight_layout.png")

In [ ]:
p_values = [0.15]
µ_values = [TC3inhib_network.connections.CE_to_CE.µ]
pops_to_modify = (:CE_to_CE,)
name = "Epileptic-like state"

plts = NetworkUtils.network_modifications(TC3inhib_network_modified, p_values, µ_values, pops_to_modify, name, img_path)
display(plts) #display the plts of the last p and µ of p_values and µ_values

#### Create a configuration of “epileptic-like activity”

Since our study was on the impact of the different inhibitory populations on the epileptic-like activity, we first wanted to see an epileptic-like activity. We did it by increasing the cortex to cortex connection (CE_to_CE).

In [ ]:
p_values = [0.15]
µ_values = [TC3inhib_network.connections.CE_to_CE.µ]
pops_to_modify = (:CE_to_CE,)
name = "Epileptic-like state"

plts = NetworkUtils.network_modifications(TC3inhib_network, p_values, µ_values, pops_to_modify, name, img_path)
display(plts) #display the plts of the last p and µ of p_values and µ_values

#### Explore the effect of thalamic input to pyramidal cells on the onset of “epileptic-like” activity

To do see what is the role of connection from the thalamic neurons to the pyramidal cells on the epileptic-like activity, we changed the probability of connection of the thalamic neurons to the cortical neurons (TE_to_CE).

In [ ]:
p_values = collect(0.01:0.005:0.2)
pops_to_modify = (:TE_to_CE,)
name = "Thalamic increase"

plt = NetworkUtils.plot_transition_time_vs_p(TC3inhib_network, p_values, pops_to_modify, name, img_path)
display(plt)

In [ ]:
p_values = [0.1]
µ_values = [TC3inhib_network.connections.TE_to_CE.µ]
pops_to_modify = (:TE_to_CE,)
name = "Thalamic increase"

plts = NetworkUtils.network_modifications(TC3inhib_network, p_values, µ_values, pops_to_modify, name, img_path)
display(plts) #display the plots of the last p value

## Explore the effect of different inhibition populations on the onset of “epileptic-like” activity

### PV loss

The first inhibition population we explored was PV. We tried to vary µ and p values to explore its effect. To visualize it we wanted to plot an heatmap to have a three dimention representation (µ, p and the onset time).

In [ ]:
pops_to_modify = (:PV_to_CE, :PV_to_PV, :PV_to_SST)
p_values = [0.03, 0.02, 0.01, 0.0]
µ_values = [2, 5, 10]
name = "PV loss"

tt = NetworkUtils.heatmap_transition_time_test(TC3inhib_network, p_values, µ_values, pops_to_modify, name, img_path)
transition_matrix = hcat(tt...)'

heatmap(
    string.(p_values),
    string.(μ_values),
    transition_matrix;
    xlabel = "p",
    ylabel = "μ",
    title = "CE transition time",
    colorbar_title = "Transition time (s)",
    color = :viridis,
  #  clims = (0, 3) # onset time between 0 and 3s
)

After that we focused on the probability of connection only for computational power reasons.

In [ ]:
p_values = collect(0.0:0.005:0.05)
pops_to_modify = (:PV_to_CE, :PV_to_PV, :PV_to_SST)
name = "PV loss"

plt = NetworkUtils.plot_transition_time_vs_p(TC3inhib_network, p_values, pops_to_modify, name, img_path)
display(plt)

After ploting the onset time for different probability of connections, we chose one p value to plot all our analysis plot (raster plot, vecplot and firing rate plot).

In [ ]:
pops_to_modify = (:PV_to_CE, :PV_to_PV, :PV_to_SST)
p_values = [0.0]
µ_values = [10]
name = "PV loss"

plts = NetworkUtils.network_modifications(TC3inhib_network, p_values, µ_values, pops_to_modify, name, img_path)
display(plts) #display the plts of the last p and µ of p_values and µ_values

### SST loss

We then did the same with the SST population.

In [ ]:
p_values = collect(0.0:0.005:0.5)
pops_to_modify = (:SST_to_CE, :SST_to_PV, :SST_to_VIP)
name = "SST loss"

plt = NetworkUtils.plot_transition_time_vs_p(TC3inhib_network, p_values, pops_to_modify, name, img_path)
display(plt)

In [ ]:
pops_to_modify = (:SST_to_CE, :SST_to_PV, :SST_to_VIP)
p_values = [0.0]
µ_values = [10]
name = "SST loss"

plts = NetworkUtils.network_modifications(TC3inhib_network, p_values, µ_values, pops_to_modify, name, img_path)
display(plts) #display the plts of the last p and µ of p_values and µ_values

Since SST knock-out doesn't induce any epileptic-like activity we increased the connection from cortex to cortex (CE_to_CE) to see if SST does have an impact on the activity.

In [ ]:
TC3inhib_network_modified = (; TC3inhib_network..., 
    connections = (; 
        TC3inhib_network.connections..., 
        CE_to_CE = (; TC3inhib_network.connections[:CE_to_CE]..., p = 0.15)
    )
)
p_values = collect(0.0:0.005:0.5)
pops_to_modify = (:SST_to_CE, :SST_to_PV, :SST_to_VIP)
name = "SST_CE loss_act"

plt = NetworkUtils.plot_transition_time_vs_p(TC3inhib_network_modified, p_values, pops_to_modify, name, img_path)
display(plt)

In [ ]:
pops_to_modify = (:SST_to_CE, :SST_to_PV, :SST_to_VIP)
p_values = [0.0]
µ_values = [10]
name = "SST loss"

plts = NetworkUtils.network_modifications(TC3inhib_network_modified, p_values, µ_values, pops_to_modify, name, img_path)
display(plts) #display the plts of the last p and µ of p_values and µ_values

### PV and SST loss

We then studied the  deficit of both PV and SST populations using the same methodology.

In [ ]:
p_values = collect(0.0:0.005:0.05)
pops_to_modify = (:PV_to_CE, :PV_to_PV, :PV_to_SST, :SST_to_CE, :SST_to_PV, :SST_to_VIP)
name = "PV_SST loss"

plt = NetworkUtils.plot_transition_time_vs_p(TC3inhib_network, p_values, pops_to_modify, name, img_path)
display(plt)

In [ ]:
pops_to_modify = (:PV_to_CE, :PV_to_PV, :PV_to_SST, :SST_to_CE, :SST_to_PV, :SST_to_VIP)
p_values = [0.03, 0.02, 0.01, 0.0]
µ_values = [2, 5, 10]
name = "PV_SST loss"

plts = NetworkUtils.network_modifications(TC3inhib_network, p_values, µ_values, pops_to_modify, name, img_path)
display(plts) #display the plts of the last p and µ of p_values and µ_values

### VIP loss

Finally, we proceed the same with VIP neurons.

In [ ]:
p_values = collect(0.0:0.005:0.05)
pops_to_modify = (:VIP_to_SST,)
name = "VIP loss"

plt = NetworkUtils.plot_transition_time_vs_p(TC3inhib_network, p_values, pops_to_modify, name, img_path)
display(plt)

In [ ]:
p_values = [0.0]
µ_values = [10]
pops_to_modify = (:VIP_to_SST,)
name = "VIP loss"

plts = NetworkUtils.network_modifications(TC3inhib_network, p_values, µ_values, pops_to_modify, name, img_path)
display(plts)

Since we didn't see any onset of the epileptic-like activity we tried to vary the cortex to cortex connection to see the VIP role on this onset.

In [ ]:
p_values = collect(0.1:0.005:0.3)
pops_to_modify = (:CE_to_CE,)
name = "VIP_CE loss_act  p=0.3"

plt = NetworkUtils.plot_transition_time_vs_p(TC3inhib_network, p_values, pops_to_modify, name, img_path)
display(plt)

In [ ]:
TC3inhib_network_modified = (; TC3inhib_network..., 
    connections = (; 
        TC3inhib_network.connections..., 
        VIP_to_SST = (; TC3inhib_network.connections[:VIP_to_SST]..., p = 0.0)
    )
)
p_values = collect(0.1:0.005:0.3)
pops_to_modify = (:CE_to_CE,)
name = "VIP_CE loss_act p=0.0"

plt = NetworkUtils.plot_transition_time_vs_p(TC3inhib_network_modified, p_values, pops_to_modify, name, img_path)
display(plt)

In [ ]:
TC3inhib_network_modified = (; TC3inhib_network..., 
    connections = (; 
        TC3inhib_network.connections..., 
        VIP_to_SST = (; TC3inhib_network.connections[:VIP_to_SST]..., p = 0.6)
    )
)
p_values = collect(0.1:0.005:0.3)
pops_to_modify = (:CE_to_CE,)
name = "VIP_CE loss_act p=0.6"

plt = NetworkUtils.plot_transition_time_vs_p(TC3inhib_network_modified, p_values, pops_to_modify, name, img_path)
display(plt)